In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib
from scipy.stats import skew
from scipy.stats.stats import pearsonr

In [3]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [4]:
path = "/Users/angshumanchakraborty/Desktop/instacart-market-basket-analysis/"

In [5]:
orders = pd.read_csv(path + 'orders.csv')
products = pd.read_csv(path+ 'products.csv')
order_products_train = pd.read_csv(path+ 'order_products__train.csv')
order_products_prior = pd.read_csv(path+ 'order_products__prior.csv')
aisles = pd.read_csv(path+ 'aisles.csv')
departments = pd.read_csv(path+ 'departments.csv')

In [6]:
## days since prior order is the only missing column
orders.fillna(value = 0, inplace = True)

In [7]:
prior_orders_data = order_products_prior.sort_values(by=['order_id'])
prior_orders_data = pd.merge(left = prior_orders_data, right = products,
                             left_on='product_id', right_on='product_id').sort_values(by=['order_id']).reset_index(drop=True)
prior_orders_data = pd.merge(left = prior_orders_data, right = aisles,
                             left_on='aisle_id', right_on='aisle_id').sort_values(by=['order_id']).reset_index(drop=True)
prior_orders_data = pd.merge(left = prior_orders_data, right = departments,
                             left_on='department_id', right_on='department_id').sort_values(by=['order_id']).reset_index(drop=True)
prior_orders_data = pd.merge(left = prior_orders_data, right = orders,
                             left_on='order_id', right_on='order_id').sort_values(by=['order_id']).reset_index(drop=True)


In [8]:
print("Shape of prior_orders_data :", prior_orders_data.shape)
prior_orders_data.head()

Shape of prior_orders_data : (32434489, 15)


,order_id,product_id,add_to_cart_order,reordered,product_name,aisle_id,department_id,aisle,department,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order
0,2,33120,1,1,Organic Egg Whites,86,16,eggs,dairy eggs,202279,prior,3,5,9,8.0
1,2,1819,8,1,All Natural No Stir Creamy Almond Butter,88,13,spreads,pantry,202279,prior,3,5,9,8.0
2,2,45918,4,1,Coconut Butter,19,13,oils vinegars,pantry,202279,prior,3,5,9,8.0
3,2,9327,3,0,Garlic Powder,104,13,spices seasonings,pantry,202279,prior,3,5,9,8.0
4,2,17794,6,1,Carrots,83,4,fresh vegetables,produce,202279,prior,3,5,9,8.0


In [9]:
del orders
del order_products_prior
del aisles
del departments

### PRODUCT FEATURES

In [10]:
## feat_1 : Number of purchases for each product
product_features = pd.DataFrame(
    {'prod_total_purchases':prior_orders_data.groupby('product_id')['order_id'].count()}).reset_index()


In [11]:
product_features

,product_id,prod_total_purchases
0,1,1852
1,2,90
2,3,277
3,4,329
4,5,15
...,...,...
49672,49684,9
49673,49685,49
49674,49686,120
49675,49687,13


In [12]:
## feat_2 : product_reorder_ratio
p_reorder = pd.DataFrame(
    {'prod_reorder_ratio':prior_orders_data.groupby('product_id')['reordered'].mean()}).reset_index()
p_reorder

,product_id,prod_reorder_ratio
0,1,0.613391
1,2,0.133333
2,3,0.732852
3,4,0.446809
4,5,0.600000
...,...,...
49672,49684,0.111111
49673,49685,0.122449
49674,49686,0.700000
49675,49687,0.461538


In [13]:
product_features = product_features.merge(p_reorder,on='product_id',how='inner')
product_features.sort_values(by = 'product_id', inplace = True)
del p_reorder

In [14]:
## feat_3: get mean position of product in the cart, sort by product_name and add to feat_2 of product_features
mean_position = pd.DataFrame(
    {'mean_position':prior_orders_data.groupby('product_id')['add_to_cart_order'].mean()}).reset_index()
mean_position.sort_values(by = 'product_id', inplace = True)

In [15]:
product_features['avg_pos_in_cart'] = mean_position['mean_position']

In [16]:
## feat_4 : aisle_reorder_rate
aisle_reorder_rate = pd.DataFrame(
    {'aisle_reorder_rate':prior_orders_data.groupby('aisle_id')['reordered'].mean()}).reset_index()
aisle_reorder_rate.sort_values(by = 'aisle_id', inplace = True)

In [17]:
temp_df = prior_orders_data.merge(aisle_reorder_rate,on='aisle_id',how='inner')
temp_df = temp_df[['product_id','aisle_id','department_id','aisle_reorder_rate']]
temp_df.drop_duplicates(inplace=True)
temp_df.reset_index(inplace=True,drop=True)
temp_df = temp_df.sort_values(by = 'product_id')
temp_df

,product_id,aisle_id,department_id,aisle_reorder_rate
10597,1,61,19,0.548698
1214,2,104,13,0.152391
34646,3,94,7,0.527615
21997,4,38,1,0.556655
37929,5,5,13,0.280627
...,...,...,...,...
25810,49684,124,5,0.572344
31524,49685,42,1,0.542171
4322,49686,112,3,0.670168
46546,49687,41,8,0.620883


In [18]:
product_features = product_features.merge(temp_df,on='product_id',how='left') 

In [19]:
del temp_df

In [20]:
## feat_5: dept reorder rate
dept_reorder_rate = pd.DataFrame(
    {'dept_reorder_rate':prior_orders_data.groupby('department_id')['reordered'].mean()}).reset_index()
dept_reorder_rate.sort_values(by = 'department_id', inplace = True)

In [21]:
product_features = product_features.merge(dept_reorder_rate,on='department_id',how='left') 
product_features

,product_id,prod_total_purchases,prod_reorder_ratio,avg_pos_in_cart,aisle_id,department_id,aisle_reorder_rate,dept_reorder_rate
0,1,1852,0.613391,5.801836,61,19,0.548698,0.574180
1,2,90,0.133333,9.888889,104,13,0.152391,0.346721
2,3,277,0.732852,6.415162,94,7,0.527615,0.653460
3,4,329,0.446809,9.507599,38,1,0.556655,0.541885
4,5,15,0.600000,6.466667,5,13,0.280627,0.346721
...,...,...,...,...,...,...,...,...
49672,49684,9,0.111111,4.333333,124,5,0.572344,0.569924
49673,49685,49,0.122449,9.571429,42,1,0.542171,0.541885
49674,49686,120,0.700000,7.500000,112,3,0.670168,0.628141
49675,49687,13,0.461538,7.538462,41,8,0.620883,0.601285


In [22]:
del dept_reorder_rate

In [23]:
product_features.to_csv(
    '/Users/angshumanchakraborty/Desktop/instacart-market-basket-analysis/Output/prod_feat.csv')

## USER FEATURES

In [24]:
## feat_1 : Get count of all products ordered by user
user_features = pd.DataFrame(
    {'user_total_purchases':prior_orders_data.groupby('user_id')['product_id'].count()}).reset_index()
user_features.sort_values(by = "user_id",inplace=True)

In [25]:
## feat_2 : Get count of all unique products for every user
uniq_user_prod = pd.DataFrame(
    {'user_unique_products':prior_orders_data.groupby('user_id')['product_id'].nunique()}).reset_index()
uniq_user_prod.sort_values(by = "user_id",inplace=True)

In [26]:
user_features["user_unique_products"] = uniq_user_prod["user_unique_products"]

In [27]:
del uniq_user_prod

In [28]:
## feat_3 : # Number of orders per customer
user_orders = prior_orders_data.groupby('user_id')['order_number'].max().to_frame('u_total_orders').reset_index()
user_orders.sort_values(by = "user_id",inplace=True)

In [29]:
user_features["user_total_orders"] = user_orders["u_total_orders"]

In [30]:
del user_orders

In [31]:
## feat_3: How frequent a customer has reordered products
u_reorder = prior_orders_data.groupby('user_id')['reordered'].mean().to_frame('u_reordered_ratio').reset_index()
u_reorder.sort_values(by = "user_id",inplace=True)

In [32]:
user_features["user_reorder_ratio"] = u_reorder["u_reordered_ratio"]

In [33]:
del u_reorder

In [34]:
## feat_4 : average cart size of user
avg_cs = prior_orders_data.groupby(
    ["user_id","order_id"])['add_to_cart_order'].count().reset_index(
    name='cart_size').groupby('user_id')['cart_size'].mean().reset_index()
avg_cs.sort_values(by = "user_id",inplace=True)

In [35]:
user_features["cart_size"] = avg_cs["cart_size"]

In [36]:
del avg_cs

In [37]:
#Get average days between 2 orders for every user
avg_days = prior_orders_data.groupby(
    ["user_id","order_id"])['days_since_prior_order'].max().reset_index(
    name='mean_days_between_orders').groupby('user_id')['mean_days_between_orders'].mean().reset_index()

In [38]:
avg_days.sort_values(by = "user_id",inplace=True)

In [39]:
user_features["mean_days_between_orders"] = avg_days["mean_days_between_orders"]

In [40]:
user_features

,user_id,user_total_purchases,user_unique_products,user_total_orders,user_reorder_ratio,cart_size,mean_days_between_orders
0,1,59,18,10,0.694915,5.900000,17.600000
1,2,195,102,14,0.476923,13.928571,14.142857
2,3,88,33,12,0.625000,7.333333,11.083333
3,4,18,17,5,0.055556,3.600000,11.000000
4,5,37,23,4,0.378378,9.250000,10.000000
...,...,...,...,...,...,...,...
206204,206205,32,24,3,0.250000,10.666667,13.333333
206205,206206,285,150,67,0.473684,4.253731,3.716418
206206,206207,223,92,16,0.587444,13.937500,13.437500
206207,206208,677,198,49,0.707533,13.816327,7.285714


In [41]:
user_features.to_csv(
'/Users/angshumanchakraborty/Desktop/instacart-market-basket-analysis/Output/user_feat.csv')

## USER-PRODUCT FEATURES

In [42]:
## feat_1 : User-product combination number of times bought
uxp_features = prior_orders_data.groupby(
    ['user_id', 'product_id'])['order_id'].count().to_frame('uxp_total_bought')
uxp_features = uxp_features.reset_index()
uxp_features.head()

,user_id,product_id,uxp_total_bought
0,1,196,10
1,1,10258,9
2,1,10326,1
3,1,12427,10
4,1,13032,3


In [43]:
## feat_2: How frequently user ordered the product ?
## # of times user ordered the product/ # of times user placed an order

user_tot_ord = prior_orders_data.groupby(["user_id"]).size().to_frame('user_tot_ord')
user_tot_ord.reset_index(inplace=True)
uxp_features = uxp_features.merge(user_tot_ord,on = 'user_id',how='left')
uxp_features['u_p_order_rate'] = uxp_features['uxp_total_bought']/uxp_features['user_tot_ord']
uxp_features.drop(columns = ['user_tot_ord'],inplace=True)

In [44]:
del user_tot_ord

In [46]:
## feat_3 : How frequently user reordered the product ?
## # of times user reordered the product/ # of times user ordered the product
df = prior_orders_data[prior_orders_data["reordered"]==1].groupby(["user_id","product_id"])["reordered"].size()
df = df/prior_orders_data.groupby(["user_id","product_id"]).size()
df = df.reset_index(name = 'reorder_rate')
df.fillna(0. , inplace = True)
uxp_features["u_p_reorder_rate"] = df["reorder_rate"]

In [47]:
## feat_4: Average position of product in the cart on orders placed by user ?    
df = prior_orders_data.groupby(
    ["user_id","product_id"])['add_to_cart_order'].mean().reset_index(name = 'mean_position')
uxp_features['u_p_avg_position'] = df['mean_position']

In [48]:
## feat_5 : Number of orders placed since the product was last ordered ?
# retrieve the last order_number placed by user-df_2, subtract with last order_number with the product in cart-df 
df = prior_orders_data.groupby(["user_id","product_id"])['order_number'].max().reset_index()
df_2 = prior_orders_data.groupby(["user_id"])['order_number'].max().reset_index()
df = df.merge(df_2,on='user_id',how='left')
df['order_diff'] = df['order_number_y'] - df['order_number_x']
uxp_features['u_p_orders_since_last'] = df['order_diff']

In [49]:
## feat_6 : Number of orders where user continuously brought a product without miss
df = prior_orders_data.groupby(["user_id","product_id"])['reordered'].apply(list).reset_index(name = 'max_streak')

In [50]:
def max_streak(row):
    maxm = 0
    summ = 0
    for i in row:
        if i==1:
            summ += 1
        else:
            if summ > maxm:
                maxm = summ
            summ = 0 
    return maxm

In [51]:
df['max_streak'] = df['max_streak'].apply(max_streak)

In [52]:
uxp_features = pd.merge(uxp_features, df, on= ["user_id","product_id"])
uxp_features

,user_id,product_id,uxp_total_bought,u_p_order_rate,u_p_reorder_rate,u_p_avg_position,u_p_orders_since_last,max_streak
0,1,196,10,0.169492,0.900000,1.400000,0,6
1,1,10258,9,0.152542,0.888889,3.333333,0,5
2,1,10326,1,0.016949,0.000000,5.000000,5,0
3,1,12427,10,0.169492,0.900000,3.300000,0,6
4,1,13032,3,0.050847,0.666667,6.333333,0,1
...,...,...,...,...,...,...,...,...
13307948,206209,43961,3,0.023256,0.666667,8.000000,1,1
13307949,206209,44325,1,0.007752,0.000000,8.000000,6,0
13307950,206209,48370,1,0.007752,0.000000,8.000000,2,0
13307951,206209,48697,1,0.007752,0.000000,6.000000,6,0


In [53]:
del df
del df_2

In [54]:
## feat_7 : Last 5 orders are only looked for checking if a product will be reordered
prior_orders_data['order_number_back'] = prior_orders_data.groupby(
    'user_id')['order_number'].transform(max) - prior_orders_data.order_number +1 

In [55]:
op5 = prior_orders_data[prior_orders_data.order_number_back <= 5]
last_five = op5.groupby(['user_id','product_id'])[['order_id']].count()
last_five.columns = ['times_last5']
last_five['times_last5_ratio'] = last_five.times_last5 / 5
last_five.reset_index(inplace=True)

In [56]:
uxp_features = uxp_features.merge(last_five,on=['user_id', 'product_id'], how='left')

In [57]:
uxp_features['times_last5'] = uxp_features['times_last5'].fillna(value=0)
uxp_features['times_last5_ratio'] = uxp_features['times_last5_ratio'].fillna(value=0)
uxp_features.head()

,user_id,product_id,uxp_total_bought,u_p_order_rate,u_p_reorder_rate,u_p_avg_position,u_p_orders_since_last,max_streak,times_last5,times_last5_ratio
0,1,196,10,0.169492,0.900000,1.400000,0,6,5.0,1.0
1,1,10258,9,0.152542,0.888889,3.333333,0,5,5.0,1.0
2,1,10326,1,0.016949,0.000000,5.000000,5,0,0.0,0.0
3,1,12427,10,0.169492,0.900000,3.300000,0,6,5.0,1.0
4,1,13032,3,0.050847,0.666667,6.333333,0,1,2.0,0.4


In [58]:
del op5
del last_five

In [59]:
uxp_features.to_csv(
'/Users/angshumanchakraborty/Desktop/instacart-market-basket-analysis/Output/uxp_feat.csv')

## TIME-BASED FEATURES

In [301]:
## feat_1: how frequently product was reordered on any given hour
hour_reorder_rate = pd.DataFrame({'hour_reorder_rate':prior_orders_data.groupby(
    ['product_id','order_hour_of_day'])["reordered"].mean()}).reset_index()

In [304]:
## feat_2: how frequently product was reordered on any given day 
day_reorder_rate = pd.DataFrame({'day_reorder_rate':prior_orders_data.groupby(
    ['product_id','order_dow'])["reordered"].mean()}).reset_index()


In [307]:
## feat_3: how frequently a product was ordered for a given difference b/w 2 orders containing product in days
p_days_since_prior_order_reorder_rate = prior_orders_data.groupby(
    ['product_id','days_since_prior_order'])["reordered"].size()
p_days_since_prior_order_reorder_rate = p_days_since_prior_order_reorder_rate/prior_orders_data.groupby(
    ["product_id"]).size()
p_days_since_prior_order_reorder_rate = p_days_since_prior_order_reorder_rate.reset_index(
    name = 'p_days_since_prior_order_reorder_rate')
p_days_since_prior_order_reorder_rate

,product_id,days_since_prior_order,p_days_since_prior_order_reorder_rate
0,1,0.0,0.102052
1,1,1.0,0.051836
2,1,2.0,0.048596
3,1,3.0,0.055616
4,1,4.0,0.041037
...,...,...,...
978520,49688,23.0,0.022472
978521,49688,24.0,0.022472
978522,49688,25.0,0.011236
978523,49688,26.0,0.022472


In [315]:
## feat_4: how frequently user ordered any product given difference between 2 orders in days ?

u_days_since_prior_order_reorder_rate = prior_orders_data.groupby(
    ['user_id','days_since_prior_order'])["reordered"].size()
u_days_since_prior_order_reorder_rate = u_days_since_prior_order_reorder_rate/prior_orders_data.groupby(
    ["user_id"]).size()
u_days_since_prior_order_reorder_rate = u_days_since_prior_order_reorder_rate.reset_index(
    name = 'u_days_since_prior_order_reorder_rate')
u_days_since_prior_order_reorder_rate

,user_id,days_since_prior_order,u_days_since_prior_order_reorder_rate
0,1,0.0,0.186441
1,1,14.0,0.101695
2,1,15.0,0.101695
3,1,19.0,0.067797
4,1,20.0,0.084746
...,...,...,...
1660732,206209,9.0,0.077519
1660733,206209,18.0,0.155039
1660734,206209,22.0,0.116279
1660735,206209,29.0,0.069767


In [318]:
## feat_5: how frequently user reordered a particular product given difference between 2 orders in days
days_since_prior_reorder_rate = prior_orders_data.groupby(
    ["user_id","product_id","days_since_prior_order"])["reordered"].size()
days_since_prior_reorder_rate = days_since_prior_reorder_rate/prior_orders_data.groupby(
    ["user_id","product_id"]).size()
days_since_prior_reorder_rate = days_since_prior_reorder_rate.reset_index(
    name = 'days_since_prior_reorder_rate')
days_since_prior_reorder_rate

## Merge all features to generate train data

In [319]:
user_features

,user_id,user_total_purchases,user_unique_products,user_total_orders,user_reorder_ratio,cart_size,mean_days_between_orders
0,1,59,18,10,0.694915,5.900000,17.600000
1,2,195,102,14,0.476923,13.928571,14.142857
2,3,88,33,12,0.625000,7.333333,11.083333
3,4,18,17,5,0.055556,3.600000,11.000000
4,5,37,23,4,0.378378,9.250000,10.000000
...,...,...,...,...,...,...,...
206204,206205,32,24,3,0.250000,10.666667,13.333333
206205,206206,285,150,67,0.473684,4.253731,3.716418
206206,206207,223,92,16,0.587444,13.937500,13.437500
206207,206208,677,198,49,0.707533,13.816327,7.285714


In [27]:
merged_df = pd.merge(
    uxp_features, user_features,  how='left', on=['user_id'])

In [28]:
merged_df = pd.merge(
    merged_df, product_features,  how='left', on=['product_id'])

In [31]:
merged_df.shape

(13307953, 23)

In [37]:
orders_train = orders[orders['eval_set']=='train']
orders_test = orders[orders['eval_set']=='test']

In [38]:
del orders

In [62]:
## No need to merge with products, aisles and departments since we need them later in features 
## (unless time based features are used)
## Can be simply merged as done in the EDA notebook

In [50]:

train_order_data = order_products_train.sort_values(by=['order_id'])
train_order_data = pd.merge(left = train_order_data, right = products,
                             left_on='product_id', right_on='product_id').sort_values(
    by=['order_id']).reset_index(drop=True)
train_order_data = pd.merge(
    left = train_order_data, right = aisles,
                             left_on='aisle_id', right_on='aisle_id').sort_values(
    by=['order_id']).reset_index(drop=True)
train_order_data = pd.merge(left = train_order_data, right = departments,
                             left_on='department_id', right_on='department_id').sort_values(by=['order_id']).reset_index(drop=True)
train_order_data = pd.merge(left = train_order_data, right = orders,
                             left_on='order_id', right_on='order_id').sort_values(by=['order_id']).reset_index(drop=True)


In [52]:
col_order = ['user_id',
 'order_id',
 'product_id',
  'aisle_id',
 'department_id',
 'add_to_cart_order',
 'reordered',
 'product_name',
 'aisle',
 'department',
 'eval_set',
 'order_number',
 'order_dow',
 'order_hour_of_day',
 'days_since_prior_order']

train_order_data = train_order_data[col_order]
print("Shape of train_order_data :", train_order_data.shape)
train_order_data.head()

Shape of train_order_data : (1384617, 15)


,user_id,order_id,product_id,aisle_id,department_id,add_to_cart_order,reordered,product_name,aisle,department,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order
0,112108,1,49302,120,16,1,1,Bulgarian Yogurt,yogurt,dairy eggs,train,4,4,10,9.0
1,112108,1,13176,24,4,6,0,Bag of Organic Bananas,fresh fruits,produce,train,4,4,10,9.0
2,112108,1,47209,24,4,7,0,Organic Hass Avocado,fresh fruits,produce,train,4,4,10,9.0
3,112108,1,10246,83,4,3,0,Organic Celery Hearts,fresh vegetables,produce,train,4,4,10,9.0
4,112108,1,43633,95,15,5,1,Lightly Smoked Sardines in Olive Oil,canned meat seafood,canned goods,train,4,4,10,9.0


In [53]:
upd_train_orders = train_order_data[['user_id','order_id','product_id','reordered']]
last_orders = upd_train_orders.groupby(['user_id'])['order_id'].max().reset_index(name = 'new_order_id')
order_details = train_order_data[['order_id','order_dow','order_hour_of_day','days_since_prior_order']]
order_details = order_details.drop_duplicates()

In [54]:
del train_order_data

In [55]:
upd_train_orders

,user_id,order_id,product_id,reordered
0,112108,1,49302,1
1,112108,1,13176,0
2,112108,1,47209,0
3,112108,1,10246,0
4,112108,1,43633,1
...,...,...,...,...
1384612,169679,3421063,14233,1
1384613,169679,3421063,13565,1
1384614,139822,3421070,35951,1
1384615,139822,3421070,16953,1


In [56]:
train_orders_merged_df = pd.merge(
    merged_df, upd_train_orders,how='left', left_on=['user_id','product_id'], right_on = ['user_id','product_id'])

In [57]:
del upd_train_orders

In [58]:
train_orders_merged_df = pd.merge(train_orders_merged_df, last_orders, on = 'user_id')

In [60]:
del last_orders

In [62]:
train_orders_merged_df.drop("order_id", axis = 1, inplace = True)
train_orders_merged_df.rename(columns = {'new_order_id':'order_id'}, inplace = True) 

In [63]:
#merge latest orders (prior_last_orders) with above user and product feat
train_orders_merged_df = pd.merge(train_orders_merged_df, order_details, on = 'order_id')
train_orders_merged_df[['reordered']]= train_orders_merged_df[['reordered']].fillna(value=0.0)

In [64]:
## Miscellaneous features not merged since Feature engineered data is already too big
# #merge with misc features

# train_orders_merged_df = pd.merge(train_orders_merged_df, hour_reorder_rate, on=['product_id','order_hour_of_day'], how = 'left')
# train_orders_merged_df[['hour_reorder_rate']]= train_orders_merged_df[['hour_reorder_rate']].fillna(value=0.0)

# train_orders_merged_df = pd.merge(train_orders_merged_df, day_reorder_rate, on=['product_id','order_dow'], how = 'left')
# train_orders_merged_df[['day_reorder_rate']]= train_orders_merged_df[['day_reorder_rate']].fillna(value=0.0)

# train_orders_merged_df = pd.merge(train_orders_merged_df, p_days_since_prior_order_reorder_rate, on=['product_id','days_since_prior_order'], how = 'left')
# train_orders_merged_df[['p_days_since_prior_order_reorder_rate']]= train_orders_merged_df[['p_days_since_prior_order_reorder_rate']].fillna(value=0.0)

# train_orders_merged_df = pd.merge(train_orders_merged_df, u_days_since_prior_order_reorder_rate, on=['user_id','days_since_prior_order'], how = 'left')
# train_orders_merged_df[['u_days_since_prior_order_reorder_rate']]= train_orders_merged_df[['u_days_since_prior_order_reorder_rate']].fillna(value=0.0)

# train_orders_merged_df = pd.merge(train_orders_merged_df, days_since_prior_reorder_rate, on=["user_id","product_id",'days_since_prior_order'], how = 'left')
# train_orders_merged_df[['days_since_prior_reorder_rate']]= train_orders_merged_df[['days_since_prior_reorder_rate']].fillna(value=0.0)

In [65]:
train_orders_merged_df.to_csv(
'/Users/angshumanchakraborty/Desktop/instacart-market-basket-analysis/Output/train_data_mod')

In [68]:
train_orders_merged_df = train_orders_merged_df.astype(
    {"order_hour_of_day":'category', "order_dow":'category'})

In [69]:
train_orders_merged_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8474661 entries, 0 to 8474660
Data columns (total 28 columns):
 #   Column                    Dtype   
---  ------                    -----   
 0   user_id                   int64   
 1   product_id                int64   
 2   uxp_total_bought          int64   
 3   u_p_order_rate            float64 
 4   u_p_reorder_rate          float64 
 5   u_p_avg_position          float64 
 6   u_p_orders_since_last     int64   
 7   max_streak                int64   
 8   times_last5               float64 
 9   times_last5_ratio         float64 
 10  user_total_purchases      int64   
 11  user_unique_products      int64   
 12  user_total_orders         int64   
 13  user_reorder_ratio        float64 
 14  cart_size                 float64 
 15  mean_days_between_orders  float64 
 16  prod_total_purchases      int64   
 17  prod_reorder_ratio        float64 
 18  avg_pos_in_cart           float64 
 19  aisle_id                  int64   
 20  de

In [71]:
train_orders_merged_df.columns

Index(['user_id', 'product_id', 'uxp_total_bought', 'u_p_order_rate',
       'u_p_reorder_rate', 'u_p_avg_position', 'u_p_orders_since_last',
       'max_streak', 'times_last5', 'times_last5_ratio',
       'user_total_purchases', 'user_unique_products', 'user_total_orders',
       'user_reorder_ratio', 'cart_size', 'mean_days_between_orders',
       'prod_total_purchases', 'prod_reorder_ratio', 'avg_pos_in_cart',
       'aisle_id', 'department_id', 'aisle_reorder_rate', 'dept_reorder_rate',
       'reordered', 'order_id', 'order_dow', 'order_hour_of_day',
       'days_since_prior_order'],
      dtype='object')

In [72]:
def reduce_size(train_orders_merged_df):
    train_orders_merged_df['user_id'] = train_orders_merged_df['user_id'].astype('int32')
    train_orders_merged_df['product_id'] = train_orders_merged_df['product_id'].astype('uint16')
    train_orders_merged_df['uxp_total_bought'] = train_orders_merged_df['uxp_total_bought'].astype('int16')
    train_orders_merged_df['u_p_order_rate'] = train_orders_merged_df['u_p_order_rate'].astype('float16')
    train_orders_merged_df['u_p_reorder_rate'] = train_orders_merged_df['u_p_reorder_rate'].astype('float16')
    train_orders_merged_df['u_p_avg_position'] = train_orders_merged_df['u_p_avg_position'].astype('float16')
    train_orders_merged_df['u_p_orders_since_last'] = train_orders_merged_df['u_p_orders_since_last'].astype('int8')
    train_orders_merged_df['max_streak'] = train_orders_merged_df['max_streak'].astype('int8')
    train_orders_merged_df['times_last5'] = train_orders_merged_df['times_last5'].astype('int8')
    train_orders_merged_df['times_last5_ratio'] = train_orders_merged_df['times_last5_ratio'].astype('int8')

    train_orders_merged_df['user_total_purchases'] = train_orders_merged_df['user_total_purchases'].astype('int16')
    train_orders_merged_df['user_unique_products'] = train_orders_merged_df['user_unique_products'].astype('int16')
    train_orders_merged_df['user_total_orders'] = train_orders_merged_df['user_total_orders'].astype('int16')
    train_orders_merged_df['user_reorder_ratio'] = train_orders_merged_df['user_reorder_ratio'].astype('float16')
    train_orders_merged_df['cart_size'] = train_orders_merged_df['cart_size'].astype('float16')
    train_orders_merged_df['mean_days_between_orders'] = train_orders_merged_df['mean_days_between_orders'].astype('float16')
    train_orders_merged_df['prod_total_purchases'] = train_orders_merged_df['prod_total_purchases'].astype('int16')
    train_orders_merged_df['prod_reorder_ratio'] = train_orders_merged_df['prod_reorder_ratio'].astype('float16')
    train_orders_merged_df['avg_pos_in_cart'] = train_orders_merged_df['avg_pos_in_cart'].astype('float16')

    train_orders_merged_df['aisle_id'] = train_orders_merged_df['aisle_id'].astype('uint8')
    train_orders_merged_df['department_id'] = train_orders_merged_df['department_id'].astype('uint8')
    train_orders_merged_df['aisle_reorder_rate'] = train_orders_merged_df['aisle_reorder_rate'].astype('float16')
    train_orders_merged_df['dept_reorder_rate'] = train_orders_merged_df['dept_reorder_rate'].astype('float16')
    train_orders_merged_df['order_id'] = train_orders_merged_df['order_id'].astype('int32')
    train_orders_merged_df['order_dow'] = train_orders_merged_df['order_dow'].astype('uint8')
    train_orders_merged_df['order_hour_of_day'] = train_orders_merged_df['order_hour_of_day'].astype('uint8')
    train_orders_merged_df['days_since_prior_order'] = train_orders_merged_df['days_since_prior_order'].astype('uint8')
    # train_orders_merged_df['hour_reorder_rate'] = train_orders_merged_df['hour_reorder_rate'].astype('float32')
    # train_orders_merged_df['day_reorder_rate'] = train_orders_merged_df['day_reorder_rate'].astype('float32')
    # train_orders_merged_df['p_days_since_prior_order_reorder_rate'] = train_orders_merged_df['p_days_since_prior_order_reorder_rate'].astype('float32')
    # train_orders_merged_df['u_days_since_prior_order_reorder_rate'] = train_orders_merged_df['u_days_since_prior_order_reorder_rate'].astype('float32')
    # train_orders_merged_df['days_since_prior_reorder_rate'] = train_orders_merged_df['days_since_prior_reorder_rate'].astype('float32')
    return train_orders_merged_df

In [71]:
train_orders_merged_df = reduce_size(train_orders_merged_df)

In [73]:
train_orders_merged_df[train_orders_merged_df.columns[:]].corr()['reordered'][:]

user_id                    -0.000429
product_id                  0.002786
uxp_total_bought            0.248376
u_p_order_rate              0.240855
u_p_reorder_rate            0.247972
u_p_avg_position           -0.042599
u_p_orders_since_last      -0.176801
max_streak                  0.204952
times_last5                 0.386080
times_last5_ratio           0.186677
user_total_purchases       -0.059297
user_unique_products       -0.098347
user_total_orders          -0.092544
user_reorder_ratio          0.007098
cart_size                   0.049083
mean_days_between_orders    0.059138
prod_total_purchases       -0.001839
prod_reorder_ratio          0.167353
avg_pos_in_cart            -0.129350
aisle_id                    0.002256
department_id              -0.033707
aisle_reorder_rate          0.130650
dept_reorder_rate           0.107237
reordered                   1.000000
order_id                    0.001226
order_dow                  -0.012129
order_hour_of_day          -0.005085
d

In [75]:
train_orders_merged_df.isnull().sum()

user_id                     0
product_id                  0
uxp_total_bought            0
u_p_order_rate              0
u_p_reorder_rate            0
u_p_avg_position            0
u_p_orders_since_last       0
max_streak                  0
times_last5                 0
times_last5_ratio           0
user_total_purchases        0
user_unique_products        0
user_total_orders           0
user_reorder_ratio          0
cart_size                   0
mean_days_between_orders    0
prod_total_purchases        0
prod_reorder_ratio          0
avg_pos_in_cart             0
aisle_id                    0
department_id               0
aisle_reorder_rate          0
dept_reorder_rate           0
reordered                   0
order_id                    0
order_dow                   0
order_hour_of_day           0
days_since_prior_order      0
dtype: int64

In [110]:
train_orders_merged_df.to_csv(
'/Users/angshumanchakraborty/Desktop/instacart-market-basket-analysis/Output/train_data_mod.csv')

## Generate Test Data

In [77]:
order_details_test = orders[orders['eval_set'] == 'test']
order_details_test.drop(['eval_set'], axis = 1, inplace = True)

/Users/angshumanchakraborty/opt/anaconda3/lib/python3.8/site-packages/pandas/core/frame.py:4308: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


In [90]:
# merge on user_product features, to get features on this order

test_orders_merge_df = pd.merge(order_details_test, uxp_features, on = ['user_id'], how = 'outer')
test_orders_merge_df.dropna(inplace =True)
test_orders_merge_df = pd.merge(test_orders_merge_df, user_features, on = ['user_id'])
test_orders_merge_df = pd.merge(test_orders_merge_df, product_features, on = ['product_id'])

In [91]:
#merge with misc features

# test_orders_merge_df = pd.merge(test_orders_merge_df, hour_reorder_rate, on=['product_id','order_hour_of_day'], how = 'left')
# test_orders_merge_df[['hour_reorder_rate']]= test_orders_merge_df[['hour_reorder_rate']].fillna(value=0.0)

# test_orders_merge_df = pd.merge(test_orders_merge_df, day_reorder_rate, on=['product_id','order_dow'], how = 'left')
# test_orders_merge_df[['day_reorder_rate']]= test_orders_merge_df[['day_reorder_rate']].fillna(value=0.0)

# test_orders_merge_df = pd.merge(test_orders_merge_df, p_days_since_prior_order_reorder_rate, on=['product_id','days_since_prior_order'], how = 'left')
# test_orders_merge_df[['p_days_since_prior_order_reorder_rate']]= test_orders_merge_df[['p_days_since_prior_order_reorder_rate']].fillna(value=0.0)

# test_orders_merge_df = pd.merge(test_orders_merge_df, u_days_since_prior_order_reorder_rate, on=['user_id','days_since_prior_order'], how = 'left')
# test_orders_merge_df[['u_days_since_prior_order_reorder_rate']]= test_orders_merge_df[['u_days_since_prior_order_reorder_rate']].fillna(value=0.0)

# test_orders_merge_df = pd.merge(test_orders_merge_df, days_since_prior_reorder_rate, on=["user_id","product_id",'days_since_prior_order'], how = 'left')
# test_orders_merge_df[['days_since_prior_reorder_rate']]= test_orders_merge_df[['days_since_prior_reorder_rate']].fillna(value=0.0)

,order_id,user_id,order_number,order_dow,order_hour_of_day,days_since_prior_order,product_id,uxp_total_bought,u_p_order_rate,u_p_reorder_rate,...,user_reorder_ratio,cart_size,mean_days_between_orders,prod_total_purchases,prod_reorder_ratio,avg_pos_in_cart,aisle_id,department_id,aisle_reorder_rate,dept_reorder_rate
0,2774568.0,3,13.0,5.0,15.0,11.0,248,1,0.011364,0.000000,...,0.625000,7.333333,11.083333,6371,0.400251,10.620782,117,19,0.519170,0.574180
1,1356845.0,12,6.0,1.0,20.0,30.0,248,1,0.013514,0.000000,...,0.175676,14.800000,20.000000,6371,0.400251,10.620782,117,19,0.519170,0.574180
2,3073553.0,418,9.0,6.0,17.0,13.0,248,1,0.005988,0.000000,...,0.359281,20.875000,20.625000,6371,0.400251,10.620782,117,19,0.519170,0.574180
3,1490499.0,503,37.0,1.0,16.0,8.0,248,1,0.002994,0.000000,...,0.613772,9.277778,9.777778,6371,0.400251,10.620782,117,19,0.519170,0.574180
4,391588.0,720,24.0,1.0,13.0,19.0,248,3,0.009709,0.666667,...,0.618123,13.434783,13.782609,6371,0.400251,10.620782,117,19,0.519170,0.574180
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4833287,913357.0,205877,45.0,3.0,10.0,21.0,27049,1,0.001684,0.000000,...,0.501684,13.500000,7.795455,9,0.333333,10.000000,40,8,0.565053,0.601285
4833288,2268059.0,205880,14.0,3.0,17.0,1.0,25635,1,0.014286,0.000000,...,0.242857,5.384615,3.230769,7,0.571429,3.571429,33,6,0.401000,0.369229
4833289,2072901.0,205893,17.0,0.0,10.0,21.0,11873,1,0.002786,0.000000,...,0.543175,22.437500,17.125000,6,0.166667,13.833333,80,11,0.236855,0.321129
4833290,2964171.0,206131,13.0,1.0,15.0,13.0,12653,2,0.013245,0.500000,...,0.549669,12.583333,17.166667,4,0.250000,10.750000,37,1,0.492351,0.541885


In [103]:
test_orders_merge_df.drop(columns=['order_number'],inplace=True)

In [104]:
test_orders_merge_df.columns

Index(['order_id', 'user_id', 'order_dow', 'order_hour_of_day',
       'days_since_prior_order', 'product_id', 'uxp_total_bought',
       'u_p_order_rate', 'u_p_reorder_rate', 'u_p_avg_position',
       'u_p_orders_since_last', 'max_streak', 'times_last5',
       'times_last5_ratio', 'user_total_purchases', 'user_unique_products',
       'user_total_orders', 'user_reorder_ratio', 'cart_size',
       'mean_days_between_orders', 'prod_total_purchases',
       'prod_reorder_ratio', 'avg_pos_in_cart', 'aisle_id', 'department_id',
       'aisle_reorder_rate', 'dept_reorder_rate'],
      dtype='object')

In [106]:
#test_orders_merged_df = reduce_size(test_orders_merged_df)

In [107]:
test_orders_merge_df.isnull().sum()

order_id                    0
user_id                     0
order_dow                   0
order_hour_of_day           0
days_since_prior_order      0
product_id                  0
uxp_total_bought            0
u_p_order_rate              0
u_p_reorder_rate            0
u_p_avg_position            0
u_p_orders_since_last       0
max_streak                  0
times_last5                 0
times_last5_ratio           0
user_total_purchases        0
user_unique_products        0
user_total_orders           0
user_reorder_ratio          0
cart_size                   0
mean_days_between_orders    0
prod_total_purchases        0
prod_reorder_ratio          0
avg_pos_in_cart             0
aisle_id                    0
department_id               0
aisle_reorder_rate          0
dept_reorder_rate           0
dtype: int64

In [108]:
test_orders_merge_df.to_csv(
    "/Users/angshumanchakraborty/Desktop/instacart-market-basket-analysis/Output/test_data_mod.csv",index=False) 